In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
from change_detection import *

In [16]:
from change_detection import *
x = ChangeDetection('practice_data_lp%',
                    measure=True,
                    sample=True,
                    verbose=True)
x.detect_change()

practice_data_lpaliskiren
practice_data_lpamiodarone


KeyboardInterrupt: 

In [17]:
x = ChangeDetection('testing',verbose=True)
x.detect_change()

,is.nbreak,is.tfirst,is.tfirst.pknown,is.tfirst.pknown.offs,is.tfirst.offs,is.tfirst.big,is.slope.ma,is.slope.ma.prop,is.slope.ma.prop.lev,is.intlev.initlev,is.intlev.finallev,is.intlev.levd,is.intlev.levdprop
name,,,,,,,,,,,,,
0101010C0,1,35.0,35.0,35.0,35.0,35.0,-9.357648e-08,-0.007155,0.507692,1.317150e-05,7.089033e-06,6.082471e-06,0.461790
0101010F0,2,23.0,23.0,23.0,23.0,23.0,-1.830919e-08,-0.014695,0.503932,1.264287e-06,6.102989e-07,6.539879e-07,0.517278
0101010G0,2,15.0,15.0,15.0,15.0,72.0,-1.707681e-06,-0.009821,0.506635,1.746973e-04,8.031958e-05,9.437769e-05,0.540236
0101010I0,2,16.0,16.0,16.0,16.0,16.0,3.788124e-08,0.007235,0.502223,5.198075e-06,5.575211e-06,-3.771360e-07,-0.072553
0101010J0,3,15.0,15.0,15.0,15.0,55.0,-1.038058e-06,-0.025005,0.503807,4.186835e-05,4.780651e-06,3.708769e-05,0.885817
0101010L0,3,10.0,10.0,NaN,NaN,10.0,-7.814762e-06,-0.120709,0.540078,7.255520e-05,1.467643e-05,5.787877e-05,0.797721
0101010P0,3,31.0,31.0,31.0,31.0,31.0,-3.902467e-06,-0.153184,0.565526,2.937814e-05,1.775760e-06,2.760238e-05,0.939555
0101010Q0,5,15.0,15.0,NaN,NaN,79.0,4.631241e-05,14.263117,4.434804,3.467604e-06,1.391055e-05,-1.044294e-05,-3.011573
0101010R0,2,13.0,13.0,13.0,13.0,58.0,-1.319898e-06,-0.011436,0.500654,1.159369e-04,3.684641e-05,7.909047e-05,0.682186


In [3]:
from change_detection import *
x = ChangeDetection(name='desogestrel_measure',verbose=True)
x.detect_change()

,is.nbreak,is.tfirstneg,is.tfirstneg.pknown,is.tfirstneg.pknown.offs,is.tfirstneg.offs,is.tfirstneg.big,is.slope.ma,is.slope.ma.prop,is.slope.ma.prop.lev,is.intlev.initlev,is.intlev.finallev,is.intlev.levd,is.intlev.levdprop
name,,,,,,,,,,,,,
A81001,2,46.000000,46.0,46.0,46.0,46.0,-0.081600,-0.090631,0.562657,0.981948,0.401846,0.580102,0.590767
A81002,2,42.000000,42.0,42.0,42.0,42.0,-0.478759,-2.844392,0.839145,0.647076,0.076544,0.570532,0.881708
A81004,3,40.000000,40.0,40.0,40.0,40.0,-0.096405,-0.110674,0.522166,0.967481,0.044352,0.923130,0.954158
A81005,1,40.000000,40.0,40.0,40.0,40.0,-0.034794,-0.036432,0.538462,0.989830,0.085181,0.904649,0.913944
A81006,3,41.000000,41.0,41.0,41.0,41.0,-0.082525,-0.093715,0.539573,0.963124,0.198397,0.764727,0.794007
A81007,1,2.000000,2.0,2.0,2.0,2.0,-0.011682,-0.017960,0.515625,0.662121,-0.085503,0.747623,1.129134
A81008,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A81009,2,44.000000,44.0,44.0,44.0,44.0,-0.278580,-0.390173,0.645130,0.992572,0.128931,0.863641,0.870105
A81011,2,40.000000,40.0,NaN,NaN,40.0,-0.452238,-0.852081,0.976209,0.982984,0.056464,0.926519,0.942558


In [2]:
x = ChangeDetection(name='trimethoprim_measure')
x.detect_change()

C:\Users\ajwalker\Anaconda3\lib\site-packages\pandas\io\gbq.py:108: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  **kwargs)


,is.nbreak,is.tfirstneg,is.tfirstneg.pknown,is.tfirstneg.pknown.offs,is.tfirstneg.offs,is.tfirstneg.big,is.slope.ma,is.slope.ma.prop,is.slope.ma.prop.lev,is.intlev.initlev,is.intlev.finallev,is.intlev.levd,is.intlev.levdprop
name,,,,,,,,,,,,,
M81072,1,43.0,43.0,43.000000,43.000000,43.0,-0.019368,-0.028289,0.526316,0.703993,0.336008,0.367985,0.522711
B85019,1,14.0,14.0,14.000000,14.000000,14.0,-0.002984,-0.004110,0.520833,0.729062,0.585838,0.143224,0.196449
E81060,2,29.0,29.0,29.000000,29.000000,29.0,-0.013271,-0.022539,0.515152,0.602082,0.164127,0.437955,0.727401
C82016,2,42.0,42.0,42.000000,42.000000,42.0,-0.022252,-0.027348,0.550000,0.835899,0.390862,0.445037,0.532406
M84005,1,30.0,30.0,30.000000,30.000000,30.0,-0.012335,-0.018942,0.531250,0.663526,0.268805,0.394721,0.594884
C85005,1,38.0,38.0,NaN,NaN,38.0,-0.011033,-0.017701,0.541667,0.634303,0.369520,0.264783,0.417440
D81037,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D81615,1,37.0,37.0,37.000000,37.000000,37.0,-0.016654,-0.026760,0.520000,0.638992,0.222648,0.416344,0.651564
K83051,2,14.0,14.0,14.000000,14.000000,14.0,-0.003164,-0.005331,0.520833,0.596760,0.444865,0.151895,0.254533
